### Create new price list

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = 'c:/Users/balob/Documents/GITLAB/RB_BD/DATA/'

eur_usd = 1.2

In [2]:
# Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE,t.NETWORK_ID,
np.MCC + IIF(LEN(MNC)=1,'0' + MNC,MNC) MCCMNC
FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES np ON t.network_id = np.network_id
'''
df_tadig_mccmnc = pd.read_sql_query(sql_srt, cnxn)
#df_tadig['NETWORK_ID']=df_tadig['NETWORK_ID'].astype('int')
df_tadig = df_tadig_mccmnc[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','NETWORK_ID']].drop_duplicates()

df_tadig_mccmnc = df_tadig_mccmnc[['NETWORK_ID','MCCMNC']].drop_duplicates()

display(df_tadig.head())

cnxn.close()

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4.0
1,Afghanistan,Areeba/MTN,AFGAR,5.0
2,Afghanistan,Etisalat,AFG55,6.0
3,Afghanistan,Etisalat,AFGEA,6.0
4,Afghanistan,Roshan/TDCA,AFGK9,7.0


In [3]:
# Загружаем новые тарифы Partner и P4

df_partner = pd.read_csv(join(load_folder,'Partner_1_1_2019_for_tariffs.csv'),sep=';')
print('Тарифы Partner:')
display(df_partner.head())

df_p4 = pd.read_csv(join(load_folder,'P4_1_1_2019_for_tariffs.csv'),sep=';')
#df_p4[['MOC','MTC','SMS','MB']] = df_p4[['MOC','MTC','SMS','MB']]*eur_usd
print('Тарифы P4:')
display(df_p4.head())

Тарифы Partner:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
0,ALBVF,27602.0,Vodafone AL,Albania,VF,0.0439,0.0,0.0062,0.0030,1,1.0
1,ANDMA,21303.0,Andorra Telecom SAU,Andorra,NaN,0.2000,0.0,0.0300,0.2000,60,1.0
2,AIACW,36584.0,AIACW AI,Anguilla,C&W Lime,0.1606,0.0,0.0128,0.0257,60,1.0
3,ATGCW,34492.0,ATGCW AG,Antigua And Barbuda,C&W Lime,0.1500,0.0,0.0120,0.0240,60,1.0
4,ARGTM,72207.0,TELE MOV AR,Argentina,Telefonica LATAM,0.0728,0.0,0.0240,0.0180,1,1.0


Тарифы P4:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
0,Albania,ALBtelecom sh.a.,ALBEM,0.390,NaN,0.040,0.039,60,1
1,Anguilla,Cable and Wireless (Anguilla) Limited,AIACW,0.800,NaN,0.080,0.080,1,1
2,Antigua,Cable & Wireless Antigua and Barbuda Limited,ATGCW,0.800,NaN,0.080,0.080,1,1
3,Armenia,Armenia Telephone Company (ArmenTel),ARM01,0.585,NaN,0.015,0.004,60,1
4,Australia,SingTel Optus Pty Limited,AUSOP,0.055,NaN,0.008,0.008,60,1


In [4]:
df_partner[df_partner.TADIG == 'GBRVF']

,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
235,GBRVF,23415.0,Vodafone GB,United Kingdom,VF,0.0439,0.0,0.0062,0.003,1,1.0


In [6]:
# Загружаем Allowed list с необходимыми изменениями

df_al=pd.read_csv(join(load_folder,'DMI.dmi_allowed_list_export_Tue_Dec_25_2018.csv'),usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al = df_al[df_al.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al = df_al.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al = pd.merge(df_al,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al.drop('TADIG_CODE',inplace=True,axis=1)

# Add PLMN

df_add_plmn = DataFrame({'SPONSOR':['S1'],'TADIG_CODE_AL':['RUSNW'],'NETWORK_ID':[769]})
df_al = df_al.append(df_add_plmn).reset_index(drop=True)

# Change Sponsor

# Croatia -> Change to P4
df_al.loc[df_al.TADIG_CODE_AL.isin(['HRVT2','HRVVI']),'SPONSOR'] = 'S2'

# Russian Federation -> Change to Partner
df_al.loc[df_al.TADIG_CODE_AL.isin(['RUSBD','RUS01','RUSNW']),'SPONSOR'] = 'S1'

# Turkey -> Change to P4
df_al.loc[df_al.TADIG_CODE_AL.isin(['TURAC','TURIS','TURTC','TURTK']),'SPONSOR'] = 'S2'
df_al.drop(df_al[df_al.TADIG_CODE_AL=='TURTS'].index,inplace=True)

print('Allowed list:')
display(df_al.head())

Allowed list:


,NETWORK_ID,SPONSOR,TADIG_CODE_AL
0,622.0,S2,MNEMT
1,569.0,S1,MKDCC
2,234.0,S2,CZEET
3,577.0,S1,MWICT
4,439.0,S1,IDNTS


In [7]:
#Объединяем тарифы с TADIG и Allowed list

df_partner_net_id = pd.merge(df_partner,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах Partner с добавлением network_id:')
display(df_partner_net_id[df_partner_net_id.TADIG_CODE.isnull()])

df_p4_net_id = pd.merge(df_p4,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах P4 с добавлением network_id:')
display(df_p4_net_id[df_p4_net_id.TADIG_CODE.isnull()])

df_s1_s2_temp = pd.merge(df_partner_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                    df_p4_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                    how='outer',on=['NETWORK_ID'],suffixes=('_S1','_S2')).drop_duplicates()
#print('Тарифы Partner и P4:')
#display(df_s1_s2_temp.head())

df_s1_s2 = pd.merge(df_s1_s2_temp,df_tadig,how='left',on='NETWORK_ID')
#print('Тарифы Partner и P4 с TADIG:')
#display(df_s1_s2.head())

df_s1_s2_sponsor = pd.merge(df_s1_s2,df_al,how='outer',on='NETWORK_ID')
#print('Тарифы Partner и P4 с привязкой к Allowed list:')
#display(df_s1_s2_sponsor.head())

# Результирующий файл

#f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S1','MOC_S2','MTC_S1','MTC_S2','SMS_S1','SMS_S2','MB_S1','MB_S2','SPONSOR']
f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE',
          'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1',
          'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2',
          'SPONSOR','TADIG_CODE_AL','NETWORK_ID']
df_s1_s2_sponsor = df_s1_s2_sponsor[f_list].drop_duplicates()
#df_s1_s2_sponsor.to_csv(join(load_folder,'Partner_P4_Compare_1_1_2019.csv'),index=False)
print('\nТарифы Partner и P4 с привязкой к Allowed list:')
df_s1_s2_sponsor.head()

Неизвестные TADIG в тарифах Partner с добавлением network_id:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
86,INDEU,40456.0,IdeaUP IN,India,IDEA,0.1,0.0,0.06,0.02,60,10.0,NaN,NaN,NaN,NaN
87,INDIB,40570.0,IDEA Bih IN,India,IDEA,0.1,0.0,0.06,0.02,60,10.0,NaN,NaN,NaN,NaN


Неизвестные TADIG в тарифах P4 с добавлением network_id:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
137,Mozambique,MOVITEL S.A,MOZVG,0.7,NaN,0.07,0.042,60,10,NaN,NaN,NaN,NaN



Тарифы Partner и P4 с привязкой к Allowed list:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,Voice_Increment_SEC_S2,Data_Increment_KB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,Albania,Vodafone,ALBVF,0.0439,0.0,0.0062,0.0030,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ALBVF,12.0
1,Andorra,Andorra Telecom S A U,ANDMA,0.2000,0.0,0.0300,0.2000,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,60.0,1.0,0.8,NaN,0.08,0.08,1.0,1.0,S1,AIACW,20.0
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,60.0,1.0,0.8,NaN,0.08,0.08,1.0,1.0,S1,ATGCW,23.0
4,Argentina,Movistar/Telefonica,ARG01,0.0728,0.0,0.0240,0.0180,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ARG01,27.0


In [14]:
df_s1_s2_sponsor[df_s1_s2_sponsor.NETWORK_ID == 1025]

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,Voice_Increment_SEC_S2,Data_Increment_KB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID
261,United Kingdom,Vodafone Ltd,GBRVF,0.0439,0.0,0.0062,0.003,1.0,1.0,0.045,NaN,0.013,0.0063,1.0,1.0,NaN,NaN,1025.0


In [15]:
# Финальная версия с MCCMNC

df_s1_s2_sponsor_mccmnc = pd.merge(df_s1_s2_sponsor,df_tadig_mccmnc,how='left',on='NETWORK_ID')
df_s1_s2_sponsor_mccmnc.head()

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,Voice_Increment_SEC_S2,Data_Increment_KB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID,MCCMNC
0,Albania,Vodafone,ALBVF,0.0439,0.0,0.0062,0.0030,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ALBVF,12.0,27602
1,Andorra,Andorra Telecom S A U,ANDMA,0.2000,0.0,0.0300,0.2000,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,21303
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,60.0,1.0,0.8,NaN,0.08,0.08,1.0,1.0,S1,AIACW,20.0,365840
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,60.0,1.0,0.8,NaN,0.08,0.08,1.0,1.0,S1,ATGCW,23.0,344920
4,Argentina,Movistar/Telefonica,ARG01,0.0728,0.0,0.0240,0.0180,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ARG01,27.0,72207


In [16]:
df_s1_s2_sponsor_mccmnc[df_s1_s2_sponsor_mccmnc.TADIG_CODE == 'GBRVF']

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,Voice_Increment_SEC_S2,Data_Increment_KB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID,MCCMNC
960,United Kingdom,Vodafone Ltd,GBRVF,0.0439,0.0,0.0062,0.003,1.0,1.0,0.045,NaN,0.013,0.0063,1.0,1.0,NaN,NaN,1025.0,23415
961,United Kingdom,Vodafone Ltd,GBRVF,0.0439,0.0,0.0062,0.003,1.0,1.0,0.045,NaN,0.013,0.0063,1.0,1.0,NaN,NaN,1025.0,23491


In [17]:
# Проверка наличия различных тарифов для одного и того же оператора.
# Операторы определяются по данным OCS

df = df_p4_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s1 = df[df.values > 1]
display(s1)
#df_s1_s2_sponsor[df_s1_s2_sponsor.NETWORK_ID.isin(s1.index.values)]

,MOC,MTC,SMS,MB
NETWORK_ID,,,,


In [37]:
# Подготовка Combined Price Plan

df1 = df_s1_s2_sponsor_mccmnc[df_s1_s2_sponsor_mccmnc.SPONSOR=='S1']\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC'
  ,'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1','SPONSOR']].copy()

df1.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S1':'MOC','MTC_S1':'MTC','SMS_S1':'SMS','MB_S1':'DATA (MB)',
            'Voice_Increment_SEC_S1':'Voice Increment (SEC)','Data_Increment_KB_S1':'Data Increment (KB)'}
           ,axis=1,inplace=True)

df2 = df_s1_s2_sponsor_mccmnc[df_s1_s2_sponsor_mccmnc.SPONSOR=='S2']\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC'
  ,'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2','SPONSOR']].copy()

df2.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S2':'MOC','MTC_S2':'MTC','SMS_S2':'SMS','MB_S2':'DATA (MB)',
            'Voice_Increment_SEC_S2':'Voice Increment (SEC)','Data_Increment_KB_S2':'Data Increment (KB)'}
           ,axis=1,inplace=True)

df2[['MOC','MTC','SMS','DATA (MB)']] = df2[['MOC','MTC','SMS','DATA (MB)']]*eur_usd

df_combined = pd.concat([df1, df2],axis=0).drop_duplicates().sort_values(by=['Country','Operator'])

df_combined.dropna(subset=['MOC','MTC','SMS','DATA (MB)'],how='all',inplace=True)

df_combined.MTC.fillna(0,inplace=True)
#df_combined.dropna(inplace=True)

df_combined[['MOC','MTC','SMS','DATA (MB)']] = df_combined[['MOC','MTC','SMS','DATA (MB)']]*2

df_combined.loc[df_combined.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
= df_combined[df_combined.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_combined.to_csv(join(load_folder,'df_combined.csv'),index=False)

df_combined.head()

,Country,Operator,TADIG,MCCMNC,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),SPONSOR
0,Albania,Vodafone,ALBVF,27602,0.0878,0.0,0.0124,0.0060,1.0,1.0,S1
2,Anguilla,Cable and Wireless,AIACW,365840,0.3212,0.0,0.0256,0.0514,60.0,1.0,S1
3,Antigua and Barbuda,C & W,ATGCW,344920,0.3000,0.0,0.0240,0.0480,60.0,1.0,S1
4,Argentina,Movistar/Telefonica,ARG01,72207,0.1456,0.0,0.0480,0.0360,1.0,1.0,S1
5,Argentina,Movistar/Telefonica,ARG01,72210,0.1456,0.0,0.0480,0.0360,1.0,1.0,S1


In [21]:
df1[df1.TADIG == 'GBRVF']

,Country,Operator,TADIG,MCCMNC,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),SPONSOR


In [10]:
df_combined.count()

Country                  987
Operator                 987
TADIG                    987
MCCMNC                   987
MOC                      987
MTC                      987
SMS                      987
DATA (MB)                987
Voice Increment (SEC)    987
Data Increment (KB)      987
SPONSOR                  987
dtype: int64

In [27]:
# Подготовка Partner Price Plan

df_s1_price_plan = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC'
  ,'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1','SPONSOR']].drop_duplicates()

df_s1_price_plan.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S1':'MOC','MTC_S1':'MTC','SMS_S1':'SMS','MB_S1':'DATA (MB)',
            'Voice_Increment_SEC_S1':'Voice Increment (SEC)','Data_Increment_KB_S1':'Data Increment (KB)'}
           ,axis=1,inplace=True)

df_s1_price_plan.MTC.fillna(0,inplace=True)
#df_s1_price_plan.dropna(inplace=True)

df_s1_price_plan[['MOC','MTC','SMS','DATA (MB)']] = df_s1_price_plan[['MOC','MTC','SMS','DATA (MB)']]*2

df_s1_price_plan.loc[df_s1_price_plan.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
= df_s1_price_plan[df_s1_price_plan.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_s1_price_plan.to_csv(join(load_folder,'df_s1_price_plan.csv'),index=False)

df_s1_price_plan.head()

,Country,Operator,TADIG,MCCMNC,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),SPONSOR
0,Albania,Vodafone,ALBVF,27602,0.0878,0.0,0.0124,0.0060,1.0,1.0,S1
1,Andorra,Andorra Telecom S A U,ANDMA,21303,0.4000,0.0,0.0600,0.4000,60.0,1.0,NaN
2,Anguilla,Cable and Wireless,AIACW,365840,0.3212,0.0,0.0256,0.0514,60.0,1.0,S1
3,Antigua and Barbuda,C & W,ATGCW,344920,0.3000,0.0,0.0240,0.0480,60.0,1.0,S1
4,Argentina,Movistar/Telefonica,ARG01,72207,0.1456,0.0,0.0480,0.0360,1.0,1.0,S1


In [12]:
df_s1_price_plan.count()

Country                  933
Operator                 933
TADIG                    933
MCCMNC                   933
MOC                      933
MTC                      933
SMS                      933
DATA (MB)                933
Voice Increment (SEC)    933
Data Increment (KB)      933
SPONSOR                  933
dtype: int64

In [36]:
# Подготовка P4 Price Plan

df_s2_price_plan = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC'
  ,'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2','SPONSOR']].drop_duplicates()

df_s2_price_plan.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S2':'MOC','MTC_S2':'MTC','SMS_S2':'SMS','MB_S2':'DATA (MB)',
            'Voice_Increment_SEC_S2':'Voice Increment (SEC)','Data_Increment_KB_S2':'Data Increment (KB)'}
           ,axis=1,inplace=True)

df_s2_price_plan.dropna(subset=['MOC','MTC','SMS','DATA (MB)'],how='all',inplace=True)
df_s2_price_plan.MTC.fillna(0,inplace=True)

df_s2_price_plan[['MOC','MTC','SMS','DATA (MB)']] = df_s2_price_plan[['MOC','MTC','SMS','DATA (MB)']]*2

df_s2_price_plan.loc[df_s2_price_plan.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
= df_s2_price_plan[df_s2_price_plan.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_s2_price_plan.to_csv(join(load_folder,'df_s2_price_plan.csv'),index=False)

df_s2_price_plan.head()

,Country,Operator,TADIG,MCCMNC,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),SPONSOR
2,Anguilla,Cable and Wireless,AIACW,365840,1.60,0.0,0.160,0.160,1.0,1.0,S1
3,Antigua and Barbuda,C & W,ATGCW,344920,1.60,0.0,0.160,0.160,1.0,1.0,S1
8,Armenia,ArmenTel,ARM01,28301,1.17,0.0,0.030,0.008,60.0,1.0,S1
11,Australia,Singtel Optus,AUSOP,50502,0.11,0.0,0.016,0.016,60.0,1.0,S1
12,Australia,Singtel Optus,AUSOP,50590,0.11,0.0,0.016,0.016,60.0,1.0,S1


In [14]:
df_s2_price_plan.count()

Country                  859
Operator                 859
TADIG                    859
MCCMNC                   859
MOC                      859
MTC                      859
SMS                      859
DATA (MB)                859
Voice Increment (SEC)    859
Data Increment (KB)      859
SPONSOR                  859
dtype: int64

In [67]:
df_s1_s2_sponsor.to_csv(join(load_folder,'result.csv'),index=False)